In [1]:
import geoengine as ge
import geoengine.unstable as gu


In [2]:
ge.initialize("http://localhost:3030/api", admin_token="8aca8875-425a-4ef1-8ee6-cdfc62dd7525")

In [3]:
raster_layer_id = "36574dc3-560a-4b09-9d22-d5945f2b8093"
point_layer_id = "a9623a5b-b6c5-404b-bc5a-313ff72e4e75"
polygon_layer_id = "b6191257-6d61-4c6b-90a4-ebfb1b23899d"


In [4]:
raster_source = gu.workflow_operators.GdalSource(raster_layer_id)
raster_source.to_dict()


{'type': 'GdalSource',
 'params': {'data': {'type': 'internal',
   'datasetId': '36574dc3-560a-4b09-9d22-d5945f2b8093'}}}

In [5]:
workflow_id = ge.register_workflow(raster_source.to_workflow_dict())
workflow_id

c7b7c5c8-ee11-5418-bd1f-9e5889f6e04a

In [6]:
point_source = gu.workflow_operators.OgrSource(point_layer_id)
point_source.to_dict()

{'type': 'OgrSource',
 'params': {'data': {'type': 'internal',
   'datasetId': 'a9623a5b-b6c5-404b-bc5a-313ff72e4e75'},
  'attributeProjection': None,
  'attributeFilters': None}}

In [7]:
workflow_id = ge.register_workflow(point_source.to_workflow_dict())
workflow_id

69cf7aaf-e828-537e-be9b-31933120c931

In [8]:
polygon_source = gu.workflow_operators.OgrSource(polygon_layer_id)
polygon_source.to_dict()

{'type': 'OgrSource',
 'params': {'data': {'type': 'internal',
   'datasetId': 'b6191257-6d61-4c6b-90a4-ebfb1b23899d'},
  'attributeProjection': None,
  'attributeFilters': None}}

In [9]:
workflow_id = ge.register_workflow(polygon_source.to_workflow_dict())
workflow_id

2c99104b-d5d9-5cdd-a2b9-bd72d1be5b6a

In [10]:
raster_vector_join = gu.workflow_operators.RasterVectorJoin(vector_source=point_source, raster_sources=[raster_source], new_column_names=["raster_value"])
raster_vector_join.to_dict()

{'type': 'RasterVectorJoin',
 'params': {'names': ['raster_value'],
  'temporalAggregation': 'none',
  'featureAggregation': 'mean'},
 'sources': {'vector': {'type': 'OgrSource',
   'params': {'data': {'type': 'internal',
     'datasetId': 'a9623a5b-b6c5-404b-bc5a-313ff72e4e75'},
    'attributeProjection': None,
    'attributeFilters': None}},
  'rasters': [{'type': 'GdalSource',
    'params': {'data': {'type': 'internal',
      'datasetId': '36574dc3-560a-4b09-9d22-d5945f2b8093'}}}]}}

In [11]:
workflow_id = ge.register_workflow(raster_vector_join.to_workflow_dict())
workflow_id

10c5ae97-2261-5b49-a807-7507d8478985

In [12]:
point_in_polygon_filter = gu.workflow_operators.PointInPolygonFilter(point_source=raster_vector_join, polygon_source=polygon_source)
point_in_polygon_filter.to_dict()

{'type': 'PointInPolygonFilter',
 'params': {},
 'sources': {'points': {'type': 'RasterVectorJoin',
   'params': {'names': ['raster_value'],
    'temporalAggregation': 'none',
    'featureAggregation': 'mean'},
   'sources': {'vector': {'type': 'OgrSource',
     'params': {'data': {'type': 'internal',
       'datasetId': 'a9623a5b-b6c5-404b-bc5a-313ff72e4e75'},
      'attributeProjection': None,
      'attributeFilters': None}},
    'rasters': [{'type': 'GdalSource',
      'params': {'data': {'type': 'internal',
        'datasetId': '36574dc3-560a-4b09-9d22-d5945f2b8093'}}}]}},
  'polygons': {'type': 'OgrSource',
   'params': {'data': {'type': 'internal',
     'datasetId': 'b6191257-6d61-4c6b-90a4-ebfb1b23899d'},
    'attributeProjection': None,
    'attributeFilters': None}}}}

In [13]:
workflow_id = ge.register_workflow(point_in_polygon_filter.to_workflow_dict())
workflow_id

bc909c3a-3256-5713-9fd5-d3385b281cd6

In [14]:
raster_type_conversion = gu.workflow_operators.RasterTypeConversion(source=raster_source, output_data_type="F32")
raster_type_conversion.to_dict()

{'type': 'RasterTypeConversion',
 'params': {'outputDataType': 'F32'},
 'sources': {'raster': {'type': 'GdalSource',
   'params': {'data': {'type': 'internal',
     'datasetId': '36574dc3-560a-4b09-9d22-d5945f2b8093'}}}}}

In [15]:
workflow_id = ge.register_workflow(raster_type_conversion.to_workflow_dict())
workflow_id

0506ade4-c177-500a-9c07-e5b1df9611e3

In [16]:
raster_scaling = gu.workflow_operators.RasterScaling(source=raster_type_conversion, scaling_mode='unscale')
raster_scaling.to_dict()

{'type': 'RasterScaling',
 'params': {'offsetKeyOrValue': None,
  'scaleKeyOrValue': None,
  'scalingMode': 'unscale'},
 'sources': {'raster': {'type': 'RasterTypeConversion',
   'params': {'outputDataType': 'F32'},
   'sources': {'raster': {'type': 'GdalSource',
     'params': {'data': {'type': 'internal',
       'datasetId': '36574dc3-560a-4b09-9d22-d5945f2b8093'}}}}}}}

In [17]:
workflow_id = ge.register_workflow(raster_scaling.to_workflow_dict())
workflow_id

fd4e6ced-4ccf-5f67-a895-ab53c65e919c

In [18]:
raster_projection = gu.workflow_operators.Reprojection(source=raster_scaling, target_spatial_reference="EPSG:4326")
raster_projection.to_dict()

{'type': 'Reprojection',
 'params': {'targetSpatialReference': 'EPSG:4326'},
 'sources': {'source': {'type': 'RasterScaling',
   'params': {'offsetKeyOrValue': None,
    'scaleKeyOrValue': None,
    'scalingMode': 'unscale'},
   'sources': {'raster': {'type': 'RasterTypeConversion',
     'params': {'outputDataType': 'F32'},
     'sources': {'raster': {'type': 'GdalSource',
       'params': {'data': {'type': 'internal',
         'datasetId': '36574dc3-560a-4b09-9d22-d5945f2b8093'}}}}}}}}}

In [19]:
workflow_id = ge.register_workflow(raster_projection.to_workflow_dict())
workflow_id

f6143343-781a-5369-8e8b-2cb321805f1a

In [20]:
vector_projection = gu.workflow_operators.Reprojection(source=point_source, target_spatial_reference="EPSG:4326")
vector_projection.to_dict()

{'type': 'Reprojection',
 'params': {'targetSpatialReference': 'EPSG:4326'},
 'sources': {'source': {'type': 'OgrSource',
   'params': {'data': {'type': 'internal',
     'datasetId': 'a9623a5b-b6c5-404b-bc5a-313ff72e4e75'},
    'attributeProjection': None,
    'attributeFilters': None}}}}

In [21]:
workflow_id = ge.register_workflow(vector_projection.to_workflow_dict())
workflow_id

88e92b39-89ad-54c8-b75e-b174b8db6760